In [1]:
import pickle
import pandas as pd
import numpy as np
from pyspark.sql import DataFrame
from aeon.classification.distance_based import ProximityTree, ProximityForest
import logging

from pyspark.sql import SparkSession
import os
from pyspark.sql import SparkSession
from data_ingestion import DataIngestion
from preprocessing import Preprocessor
from local_model_manager_DT import LocalModelManager_DT
from prediction_manager import PredictionManager
from local_model_manager import LocalModelManager
from evaluation import Evaluator
from utilities import show_compact
import time
import json

In [2]:
spark = SparkSession \
    .builder \
    .master("local[*]") \
    .appName("ProximityTree") \
    .getOrCreate()

sc = spark.sparkContext

In [3]:
current_dir = os.getcwd()
project_root = os.path.abspath(os.path.join(current_dir, "..", ".."))

config_DataIngestion = {
            # "data_path": project_root + "/ECG5000/*.tsv" # IF WE DECIDE TO USE .tsv FILES
            "data_path": project_root + "/fulldataset_ECG5000.csv", # IF WE DECIDE TO USE .csv FILES
            "data_percentage": 0.5  #*100% Percentage of data to load for SW development
            }   

ingestion = DataIngestion(spark,config_DataIngestion)
df = ingestion.load_data()
print(df.show(5))
df.printSchema()

Data Path is c:\Users\benat\OneDrive\0. MSc MLiS\0. GitHub Repositories\BigDataProject_repo/fulldataset_ECG5000.csv and loading 50.0% of data ++++++++++++++++++++++

Data size is :2472

+-----+-------------+-----------+----------+----------+----------+----------+----------+----------+----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+------------+------------+------------+------------+------------+------------+------------+------------+------------+-------------+-------------+-----------+------------+------------+------------+-----------+-----------+-----------+-----------+----

In [4]:
config_preproc = {}    
preprocessor = Preprocessor(config_preproc)

preprocessed_df = preprocessor.run_preprocessing(df)
print(preprocessed_df.show(5))

+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+------------------+------------------+------------------+------------------+-------------------+-------------------+-------------------+-------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-------------------+------------------+------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+------------------+------------------+------------------+-------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-------------------+------------------+------------------+------------------+------------------+-------

In [5]:
df.count()

2472

In [6]:
df.rdd.getNumPartitions() # Number of partitions in the DataFrame

2

In [15]:
for partition in range(df.rdd.getNumPartitions()):
    print(f"Number of rows in partition {partition+1}: {len(df.rdd.glom().collect()[partition])}")
    

Number of rows in partition 1: 1307
Number of rows in partition 2: 1165
